# IoT Project

##### To get started

We need the following Python libraries:  
- Scapy: For parsing network packet traces
- IPy: For each IP addressing types (what is a local network address vs what is public network address, etcc)
- manuf: Look up a device manufacturer based on MAC address data

In [51]:
!pip install scapy IPy manuf pycurl StringIO

  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO


In [3]:
from scapy.all import *
from manuf import manuf

In [11]:
#help(scapy.all)

In [39]:
packets =rdpcap("./packet_capture.pcap")
p = manuf.MacParser(update=True)
# Example for testing Manufacturer look up based on MAC ID
p.get_all('84:38:35:5a:70:40')[0]

'Apple'

In [127]:
ls(IP.proto)

Not a packet class or name. Type 'ls()' to list packet classes.


In [156]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scapy.all import *
from IPy import IP as IPy
import pprint

d= defaultdict(list)

columns = ['MAC ID', 'Manufacturer']
df = pd.DataFrame(columns=columns)

for i in range(len(packets)):
#for i in range(500):
    packet = packets[i]
    ## traffic patterns:
    # ARP: ignore for now
    # let's not worry about IPv6 for now
    if (packet.haslayer(IPv6) == False):
        # DNS
        if(packet.haslayer(DNS)):
            # look for DNS responses 
            #print(packet.summary())
            #is it DNS or mDNS traffic
            if (packet.haslayer(ICMP)):
                # this is a DNS request within ICMP
                mac_id = packet["Ether.src"]
                IP_src = packet["IP.src"]
                dns_qd_qname = str(packet[DNS].qd.qname)
                if mac_id not in d:
                    d[mac_id] = set([p.get_all(mac_id)[0], "dns_qd: " + dns_qd_qname])
                else:
                    d[mac_id].add("dns_qd: " + dns_qd_qname)
            elif (packet[UDP].sport == 5353):
                # this is mDNS
                # print(packet[DNS].an.rrname)
                #print(packet.show())
                mac_id = packet["Ether.src"]
                IP_src = packet["IP.src"]
                if (packet[DNS].an != None):
                    mdns_an_rrname = str(packet[DNS].an.rrname)
                    if mac_id not in d:
                        d[mac_id] = set([p.get_all(mac_id)[0], "mdns an: " + mdns_an_rrname])
                    else:
                        d[mac_id].add("mdns an: " + mdns_an_rrname)
                elif (packet[DNS].ns != None):
                    mdns_ns_rrname = str(packet[DNS].ns.rrname)
                    if mac_id not in d:
                        d[mac_id] = set([p.get_all(mac_id)[0], "mdns ns: " + mdns_ns_rrname])
                    else:
                        d[mac_id].add("mdns ns: " + mdns_ns_rrname)
            #elif (packet[UDP].sport == 53):
            else:
                # this is DNS
                # these packets come from the DNS server to the IoT device
                # so we are interested in the destination mac ID
                mac_id = packet["Ether.dst"]
                IP_src = packet["IP.src"]
                IP_dst = packet["IP.dst"]
                if (packet[DNS].qd != None):
                    mdns_qd_qname = str(packet[DNS].qd.qname)
                    if mac_id not in d:
                        d[mac_id] = set([p.get_all(mac_id)[0], "mdns qd: " + mdns_qd_qname])
                    else:
                        d[mac_id].add("mdns qd: " + mdns_qd_qname)
        # HTTP:
        elif(packet.haslayer(IP)):
            if (packet.haslayer(TCP)):
                # This is a terrible hack for detecting http traffic
                if (packet[TCP].dport == 80):
                    mac_id = packet["Ether.src"]
                    IP_src = packet["IP.src"]
                    IP_dst = packet["IP.dst"]
                    # is there a real payload?
                    if(len(packet[TCP].payload) > 10):
                        payload = str(packet[TCP].payload)
                        if (IPy(IP_src).iptype() == "PRIVATE"):
                            if (IPy(IP_dst).iptype() == "PUBLIC"):
                                if mac_id not in d:
                                    d[mac_id] = set([p.get_all(mac_id)[0],IP_dst, "http: " + payload])
                                else:
                                    d[mac_id].add(IP_dst +  " http: " + payload)
                    else:
                        if (IPy(IP_src).iptype() == "PRIVATE"):
                            if (IPy(IP_dst).iptype() == "PUBLIC"):
                                if mac_id not in d:
                                    d[mac_id] = set([p.get_all(mac_id)[0],IP_dst])
                                else:
                                    d[mac_id].add(IP_dst)
                        
        
    # Else: What did I miss
    
    
    # if the device is in the dataframe
    #df.loc[i] = [mac_id, p.get_all(mac_id)[0]]
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(d)

defaultdict(<class 'list'>,
            {   '28:f0:76:31:d3:58': {   'Apple',
                                         "mdns qd: b'clients6.google.com.'"},
                '40:cb:c0:bc:36:7e': {   'Apple',
                                         "mdns an: b'40CBC0BC367E@Apple "
                                         "TV._raop._tcp.local.'",
                                         'mdns an: '
                                         "b'_sleep-proxy._udp.local.'"},
                '78:28:ca:03:80:0c': {   '151.101.40.246',
                                         "151.101.40.246 http: b'GET "
                                         '/audio/7fd0fe06a28db9706f4bda8b54b1163b6dc8e6c4?1524877897_1OEE149cqh4/+6CJbhcvk2Qr2bpaxoZ8jp5KhMnkA/U= '
                                         'HTTP/1.1\\r\\nHost: '
                                         'audio-fac.spotify.com\\r\\nAccept-Encoding:\\r\\nUser-Agent: '
                                         'Sonos;One;;;2.56.45\\r\\nCache-Control:

In [151]:
lsc()

IPID_count          : Identify IP id values classes in a list of packets
arpcachepoison      : Poison target's cache with (your MAC,victim's IP) couple
arping              : Send ARP who-has requests to determine which hosts are up
bind_layers         : Bind 2 layers on some specific fields' values
bridge_and_sniff    : Forward traffic between interfaces if1 and if2, sniff and return
chexdump            :  Build a per byte hexadecimal representation
computeNIGroupAddr  : Compute the NI group Address. Can take a FQDN as input parameter
corrupt_bits        : Flip a given percentage or number of bits from a string
corrupt_bytes       : Corrupt a given percentage or number of bytes from a string
defrag              : defrag(plist) -> ([not fragmented], [defragmented],
defragment          : defrag(plist) -> plist defragmented as much as possible 
dhcp_request        : --
dyndns_add          : Send a DNS add message to a nameserver for "name" to have a new "rdata"
dyndns_del          : Send 

In [59]:
import pycurl
from io import StringIO

c = pycurl.Curl()
c.setopt(c.URL, 'https://ipinfo.io/216.58.195.243?token=4b22562280ac95')

e = io.BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = e.getvalue().decode('UTF-8')
print(htmlString)

{
  "ip": "216.58.195.243",
  "hostname": "sfo03s06-in-f19.1e100.net",
  "city": "Mountain View",
  "region": "California",
  "country": "US",
  "loc": "37.4192,-122.0570",
  "postal": "94043",
  "org": "AS15169 Google LLC"
}


In [60]:
type(htmlString)

str